In [1]:
import numpy as np
from qiskit import *
from math import pi
from math import log10
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from qiskit.circuit import *

In [3]:
n=5
k=2

In [4]:
circuit=QuantumCircuit(n+k+1)

In [5]:
circuit.h(k+1)
for i in range(k+2,n+k+1):
    circuit.cx(k+1,i)
circuit.h(1)
for i in range(2,k+1):
    circuit.cx(1,i)
circuit.barrier(range(k+1,n+k+1))

In [6]:
circuit.draw()

q_0: ────────────────────────────
     ┌───┐                       
q_1: ┤ H ├──■────────────────────
     └───┘┌─┴─┐                  
q_2: ─────┤ X ├──────────────────
     ┌───┐└───┘                ░ 
q_3: ┤ H ├──■────■────■────■───░─
     └───┘┌─┴─┐  │    │    │   ░ 
q_4: ─────┤ X ├──┼────┼────┼───░─
          └───┘┌─┴─┐  │    │   ░ 
q_5: ──────────┤ X ├──┼────┼───░─
               └───┘┌─┴─┐  │   ░ 
q_6: ───────────────┤ X ├──┼───░─
                    └───┘┌─┴─┐ ░ 
q_7: ────────────────────┤ X ├─░─
                         └───┘ ░

In [7]:
#params=[Parameter('theta'),Parameter('B'),Parameter('C')]
string="["
ng=0
for i in range(n*(n+1)-1):
    string=string+"[Parameter(\'theta_"+str(ng)+"\'),Parameter(\'phi_"+str(ng)+"\'),Parameter(\'lamda_"+str(ng)+"\')],"
    ng+=1
string=string+"[Parameter(\'theta_"+str(ng)+"\'),Parameter(\'phi_"+str(ng)+"\'),Parameter(\'lamda_"+str(ng)+"\')]]"

params=eval(string)

#print(params)

U_AB=QuantumCircuit(n,name='U_AB')
nu=0
for i in range(n):
    U_AB.u3(params[nu][0],params[nu][1],params[nu][2],i)
    nu+=1

U_AB.barrier(range(n))
    
for i in range(n):
    for j in range(n):
        if j!=i:
            U_AB.cu3(params[nu][0],params[nu][1],params[nu][2],i,j)
            nu+=1
        
U_AB.barrier(range(n))
        
for i in range(n):
    U_AB.u3(params[nu][0],params[nu][1],params[nu][2],i)
    nu+=1

U_inst=U_AB.to_instruction()
U_AB.draw()

┌───────────────────────────┐ ░                              »
q_0: ┤ U3(theta_0,phi_0,lamda_0) ├─░───────────────■──────────────»
     ├───────────────────────────┤ ░ ┌─────────────┴─────────────┐»
q_1: ┤ U3(theta_1,phi_1,lamda_1) ├─░─┤ U3(theta_5,phi_5,lamda_5) ├»
     ├───────────────────────────┤ ░ └───────────────────────────┘»
q_2: ┤ U3(theta_2,phi_2,lamda_2) ├─░──────────────────────────────»
     ├───────────────────────────┤ ░                              »
q_3: ┤ U3(theta_3,phi_3,lamda_3) ├─░──────────────────────────────»
     ├───────────────────────────┤ ░                              »
q_4: ┤ U3(theta_4,phi_4,lamda_4) ├─░──────────────────────────────»
     └───────────────────────────┘ ░                              »
«                                                               »
«q_0: ──────────────■────────────────────────────■──────────────»
«                   │                            │              »
«q_1: ──────────────┼────────────────────────────┼──────────────»
«     ┌─────────────┴─────────────┐              │              »
«q_2: ┤ U3(theta_6,phi_6,lamda_6) ├──────────────┼──────────────»
«     └───────────────────────────┘┌─────────────┴─────────────┐»
«q_3: ─────────────────────────────┤ U3(theta_7,phi_7,lamda_7) ├»
«                                  └───────────────────────────┘»
«q_4: ──────────────────────────────────────────────────────────»
«                                                               »
«                                  ┌───────────────────────────┐»
«q_0: ──────────────■──────────────┤ U3(theta_9,phi_9,lamda_9) ├»
«                   │              └─────────────┬─────────────┘»
«q_1: ──────────────┼────────────────────────────■──────────────»
«                   │                                           »
«q_2: ──────────────┼───────────────────────────────────────────»
«                   │                                           »
«q_3: ──────────────┼───────────────────────────────────────────»
«     ┌─────────────┴─────────────┐                             »
«q_4: ┤ U3(theta_8,phi_8,lamda_8) ├─────────────────────────────»
«     └───────────────────────────┘                             »
«                                                                     »
«q_0: ────────────────────────────────────────────────────────────────»
«                                                                     »
«q_1: ───────────────■───────────────────────────────■────────────────»
«     ┌──────────────┴───────────────┐               │                »
«q_2: ┤ U3(theta_10,phi_10,lamda_10) ├───────────────┼────────────────»
«     └──────────────────────────────┘┌──────────────┴───────────────┐»
«q_3: ────────────────────────────────┤ U3(theta_11,phi_11,lamda_11) ├»
«                                     └──────────────────────────────┘»
«q_4: ────────────────────────────────────────────────────────────────»
«                                                                     »
«     ┌──────────────────────────────┐                                »
«q_0: ┤ U3(theta_13,phi_13,lamda_13) ├────────────────────────────────»
«     └──────────────┬───────────────┘                                »
«q_1: ───────────────┼───────────────────────────────■────────────────»
«                    │                               │                »
«q_2: ───────────────■───────────────────────────────┼────────────────»
«                                                    │                »
«q_3: ───────────────────────────────────────────────┼────────────────»
«                                     ┌──────────────┴───────────────┐»
«q_4: ────────────────────────────────┤ U3(theta_12,phi_12,lamda_12) ├»
«                                     └──────────────────────────────┘»
«                                                                     »
«q_0: ────────────────────────────────────────────────────────────────»
«     ┌──────────────────────────────┐                                »
«q_1:

In [8]:
circuit.append(U_inst,range(k+1,n+k+1))
#circuit.draw()

In [9]:
circuit.barrier(range(2*k+1,n+k+1))
circuit.h(0)
for i in range(k):
    circuit.cswap(0,i+1,i+k+1)
circuit.h(0)
meas=QuantumCircuit(n+k+1,1)
# meas.measure(range(5,8),range(3))
meas.measure(0,0)
qc=circuit+meas
qc.draw()

┌───┐                       »
q_0: ┤ H ├───────────────────────»
     ├───┤                       »
q_1: ┤ H ├──■────────────────────»
     └───┘┌─┴─┐                  »
q_2: ─────┤ X ├──────────────────»
     ┌───┐└───┘                ░ »
q_3: ┤ H ├──■────■────■────■───░─»
     └───┘┌─┴─┐  │    │    │   ░ »
q_4: ─────┤ X ├──┼────┼────┼───░─»
          └───┘┌─┴─┐  │    │   ░ »
q_5: ──────────┤ X ├──┼────┼───░─»
               └───┘┌─┴─┐  │   ░ »
q_6: ───────────────┤ X ├──┼───░─»
                    └───┘┌─┴─┐ ░ »
q_7: ────────────────────┤ X ├─░─»
                         └───┘ ░ »
c_0: ════════════════════════════»
                                 »
«                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               »
«q_0: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
«                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               »
«q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
«                                                                                                                                                                                                                                                                                                                                                                                                   

In [10]:
def assign_params(np_array,circ=qc,parameters=params,array_name="ps"):
    string2="{"
    ngf=0
    pas=parameters
    for i in range(n*(n+1)-1):
        string2=string2+"pas["+str(ngf)+"][0]:"+array_name+"["+str(ngf)+"][0],pas["+str(ngf)+"][1]:"+array_name+"["+str(ngf)+"][1],pas["+str(ngf)+"][2]:"+array_name+"["+str(ngf)+"][2],"
        ngf+=1
    string2=string2+"pas["+str(ngf)+"][0]:"+array_name+"["+str(ngf)+"][0],pas["+str(ngf)+"][1]:"+array_name+"["+str(ngf)+"][1],pas["+str(ngf)+"][2]:"+array_name+"["+str(ngf)+"][2]}"
    dictionary=eval(string2)
    bound_circuit=circ.assign_parameters(dictionary)
    backend_sim=Aer.get_backend('qasm_simulator')
    job_sim=execute(bound_circuit,backend_sim,shots=1024)
    result_sim=job_sim.result()
    counts=result_sim.get_counts(bound_circuit)
    c2_func=2*counts['0']/1024-1
    func=log10(1-c2_func)
    return func

In [11]:
ps=np.zeros((n*(n+1),3))
f=assign_params(ps)
print(f)

-0.276300670101425


In [12]:
from scipy.optimize import basinhopping

In [13]:
minimizer_kwargs = {"method":"L-BFGS-B"}
x = basinhopping(assign_params, ps, minimizer_kwargs=minimizer_kwargs,niter=1000,disp=True)
fm=assign_params(x)
print(fm)

basinhopping step 0: f -0.294297
basinhopping step 1: f -0.314818 trial_f -0.314818 accepted 1  lowest_f -0.314818
found new global minimum on step 1 with function value -0.314818
basinhopping step 2: f -0.313071 trial_f -0.313071 accepted 1  lowest_f -0.314818
basinhopping step 3: f -0.32188 trial_f -0.32188 accepted 1  lowest_f -0.32188
found new global minimum on step 3 with function value -0.32188
basinhopping step 4: f -0.269937 trial_f -0.269937 accepted 1  lowest_f -0.32188
basinhopping step 5: f -0.279518 trial_f -0.279518 accepted 1  lowest_f -0.32188
basinhopping step 6: f -0.290969 trial_f -0.290969 accepted 1  lowest_f -0.32188
basinhopping step 7: f -0.294297 trial_f -0.294297 accepted 1  lowest_f -0.32188
basinhopping step 8: f -0.263666 trial_f -0.263666 accepted 1  lowest_f -0.32188
basinhopping step 9: f -0.304436 trial_f -0.304436 accepted 1  lowest_f -0.32188
basinhopping step 10: f -0.309596 trial_f -0.309596 accepted 1  lowest_f -0.32188
basinhopping step 11: f -0.

basinhopping step 79: f -0.286024 trial_f -0.286024 accepted 1  lowest_f -0.338202
basinhopping step 80: f -0.282759 trial_f -0.282759 accepted 1  lowest_f -0.338202
basinhopping step 81: f -0.294297 trial_f -0.294297 accepted 1  lowest_f -0.338202
basinhopping step 82: f -0.276301 trial_f -0.276301 accepted 1  lowest_f -0.338202
basinhopping step 83: f -0.286024 trial_f -0.286024 accepted 1  lowest_f -0.338202
basinhopping step 84: f -0.269937 trial_f -0.269937 accepted 1  lowest_f -0.338202
basinhopping step 85: f -0.314818 trial_f -0.314818 accepted 1  lowest_f -0.338202
basinhopping step 86: f -0.32188 trial_f -0.32188 accepted 1  lowest_f -0.338202
basinhopping step 87: f -0.282759 trial_f -0.282759 accepted 1  lowest_f -0.338202
basinhopping step 88: f -0.306149 trial_f -0.306149 accepted 1  lowest_f -0.338202
basinhopping step 89: f -0.274701 trial_f -0.274701 accepted 1  lowest_f -0.338202
basinhopping step 90: f -0.299337 trial_f -0.299337 accepted 1  lowest_f -0.338202
basinh

basinhopping step 159: f -0.290969 trial_f -0.290969 accepted 1  lowest_f -0.362917
basinhopping step 160: f -0.30103 trial_f -0.30103 accepted 1  lowest_f -0.362917
basinhopping step 161: f -0.304436 trial_f -0.304436 accepted 1  lowest_f -0.362917
basinhopping step 162: f -0.327253 trial_f -0.327253 accepted 1  lowest_f -0.362917
basinhopping step 163: f -0.281135 trial_f -0.281135 accepted 1  lowest_f -0.362917
basinhopping step 164: f -0.323664 trial_f -0.323664 accepted 1  lowest_f -0.362917
basinhopping step 165: f -0.30273 trial_f -0.30273 accepted 1  lowest_f -0.362917
basinhopping step 166: f -0.29597 trial_f -0.29597 accepted 1  lowest_f -0.362917
basinhopping step 167: f -0.29597 trial_f -0.29597 accepted 1  lowest_f -0.362917
basinhopping step 168: f -0.279518 trial_f -0.279518 accepted 1  lowest_f -0.362917
basinhopping step 169: f -0.29765 trial_f -0.29765 accepted 1  lowest_f -0.362917
basinhopping step 170: f -0.359022 trial_f -0.359022 accepted 1  lowest_f -0.362917
ba

basinhopping step 248: f -0.252904 trial_f -0.252904 accepted 1  lowest_f -0.362917
basinhopping step 249: f -0.307869 trial_f -0.307869 accepted 1  lowest_f -0.362917
adaptive stepsize: acceptance rate 0.988000 target 0.500000 new stepsize 0.846754 old stepsize 0.762079
basinhopping step 250: f -0.29597 trial_f -0.29597 accepted 1  lowest_f -0.362917
basinhopping step 251: f -0.309596 trial_f -0.309596 accepted 1  lowest_f -0.362917
basinhopping step 252: f -0.289314 trial_f -0.289314 accepted 1  lowest_f -0.362917
basinhopping step 253: f -0.292629 trial_f -0.292629 accepted 1  lowest_f -0.362917
basinhopping step 254: f -0.327253 trial_f -0.327253 accepted 1  lowest_f -0.362917
basinhopping step 255: f -0.287666 trial_f -0.287666 accepted 1  lowest_f -0.362917
basinhopping step 256: f -0.29765 trial_f -0.29765 accepted 1  lowest_f -0.362917
basinhopping step 257: f -0.279518 trial_f -0.279518 accepted 1  lowest_f -0.362917
basinhopping step 258: f -0.304436 trial_f -0.304436 accepte

KeyboardInterrupt: 